In [1]:
!pip install transformers
!pip install torch


In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('/kaggle/input/cleaneddataset/cleaned_data_v2.csv')
data = data[:1000]


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class EssayDataset(Dataset):
    def __init__(self, essays, scores, tokenizer, max_length):
        self.essays = essays
        self.scores = scores
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.essays)

    def __getitem__(self, idx):
        essay = self.essays[idx]
        score = self.scores[idx]
        encoding = self.tokenizer.encode_plus(
            essay,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'score': torch.tensor(score, dtype=torch.long),
               'score': torch.tensor(score - 1, dtype=torch.long),
        }


In [5]:
!pip install tqdm


In [6]:
import time
from tqdm import tqdm

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)
train_dataset = EssayDataset(data['essay'], data['scores'], tokenizer, max_length=512)
train_loader = DataLoader(train_dataset, batch_size=8, sampler=RandomSampler(train_dataset))
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * 3)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

for epoch in range(20):
    print(f"Epoch {epoch + 1}/{20}")
    start_time = time.time()
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader)
    for batch in progress_bar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        scores = batch['score'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=scores)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        progress_bar.set_description(f"Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f"Average Loss: {avg_loss:.4f} | Time: {epoch_time:.2f} seconds")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1/20


Loss: 0.6468: 100%|██████████| 125/125 [01:44<00:00,  1.19it/s]


Average Loss: 0.9581 | Time: 104.70 seconds
Epoch 2/20


Loss: 0.4653: 100%|██████████| 125/125 [01:45<00:00,  1.18it/s]


...

Average Loss: 0.5291 | Time: 105.96 seconds
Epoch 19/20


Loss: 0.8008: 100%|██████████| 125/125 [01:46<00:00,  1.18it/s]


Average Loss: 0.5245 | Time: 106.32 seconds
Epoch 20/20


Loss: 0.7709: 100%|██████████| 125/125 [01:46<00:00,  1.17it/s]

Average Loss: 0.5282 | Time: 106.47 seconds


In [7]:
def predict_score(model, essay, tokenizer):
    model.eval()
    encoding = tokenizer.encode_plus(
        essay,
        add_special_tokens=True,
        max_length=512,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt',
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs[0]
    score = torch.argmax(logits, dim=1).item() + 1
    return score

new_essay = "Dear @LOCATION1, I think that computers have a negative affect on us! How many people have acess to a camputer daily in america.. @NUM1 and how many people go on at least an hour a @NUM2. That means that @NUM3 people cant exercise are wasting many are have the posibility of physical @CAPS1 that sound good to you? Think of everything you done when you write a letter. I got up and got all the materials and sit back down. After In done writing have to put all the materials away and then put the letter in the mailbox and walk all the way back. Well this is what I had to do. Now think how you write an email sitdown and move your fingers. Do you see the difference? @CAPS2 instead of getting a good walk to your friends house to talk to you just in him/er. Did you know that you can literally but from on your computer. Instead of around with this that can add anything you your computer gives away information they information lead to that and then everthing you have wouldbe gone. All this so you can go online. Believe it or not, you can phisically get hurt for being on the computer, to long. @NUM4 thing is that when you at a computer @CAPS2 what happened to my grandpa is that when you are in a wood chair all day you can get ardthritis in your muscle @CAPS3 thing th."
predicted_score = predict_score(model, new_essay, tokenizer)
print("Predicted Score:", predicted_score)

Predicted Score: 4
